In [4]:
from functions import *
combine generate_adversarial_texts with analyze sections
# print("cool output file with YOU WROTE: {extracted_text} AI answered {AI_response}")
# print("pdf instead of txt for opposition")
# print("extract_from_paragraphs should have summary function so that summary is shown in output")

article_file = "data/intelligence_addon.pdf"
##OPTIONAL added_lit_file = "data/mutualism_literature.csv"

###LITERATURE
# lit_df = generate_literature_csv_from_pdf(article_file, "Author", "Title", "Publication Year", output_path="full_lit.csv")
##OPTIONAL added_lit_df = pd.read_csv(added_lit_file, keep_default_na=False, encoding="utf-8")
##OPTIONAL combined_df = pd.concat([added_lit_df, lit_df], ignore_index=True) #suggest to put df with more added abstracts first
##OPTIONAL combined_df = remove_duplicate_rows(combined_df)
##OPTIONAL combined_df.to_csv("full_lit2.csv", index=False)
# df = autosearch_missing_abstracts("full_lit2.csv", "full_lit_abstract_search2.csv")
# df = manual_entry_abstracts("full_lit_abstract_search2.csv", "full_lit_abstract_search2.csv")
# data_formatted = format_texts_from_csv("full_lit_abstract_search2.csv", "Abstract Note", "Author", "Title", "Publication Year")
# data_formatted.to_csv("trash.csv")
# vectorstore_folder_name = generate_vectorstore(data_formatted, "full_lit_abstract_search2.csv", max_doc_size = 4000)

###CLAIMS
# claims = extract_focal_claims(article_file, max_claims=3)
# sections = analyze_paragraphs(article_file, max_paragraphs=4)

###OPPOSITION
# relevant_docs = get_relevant_docs(presented_claim, vectorstore_folder_name)
# responses, summary_response = generate_adversarial_texts(presented_claim, relevant_docs)

In [5]:
sections = analyze_paragraphs(article_file, max_paragraphs=5)

In [6]:
import json
json_sections = json.dumps(sections[0], indent=3)
print(json_sections)

{
   "claim": "The central claim is that general intelligence can be conceptualized as dynamically growing networks, providing a novel understanding of the shaping of intelligence that captures its positive manifold, hierarchical structure, and opens new avenues to study its complex structure and dynamic processes at the individual level.",
   "page_content": "https://doi.org/10.1177/1745691619866447Perspectives on Psychological Science\n2019, Vol. 14(6) 1034 \u20131061\n\u00a9 The Author(s) 2019\nArticle reuse guidelines: sagepub.com/journals-permissions\nDOI: 10.1177/1745691619866447\nwww.psychologicalscience.org/PPSASSOCIATION FOR\nPSYCHOLOGICAL SCIENCE\nFormal models of intelligence have greatly evolved since \nSpearman\u2019s (1904) fundamental finding of the positive manifold: the robust pattern of positive correlations between scores on cognitive tests (Carroll, 1993). In explaining this manifold, contemporary models have diverged from the popular reflective latent factor models

In [40]:
create_pdf_from_dicts(sections, "trash.pdf")